# Tendon Steering

The body of the robot is steered using 3 tendons, evenly spaced around the body. These tendons allow for steering in all directions. They are controlled using stepper motors and spools.

## Modeling

Controlling the tendons is relatively simple. There is an equation to represent the steering from [this](https://par.nsf.gov/servlets/purl/10100241) paper:
$$
l_i=l-\theta(D_{tube}/2)cos(\phi_i)
$$

| Variable   | Description                   |
|------------|-------------------------------|
| $l_i$      | Tendon Length (meters)        |
| $l$        | Body Length (meters)          |
| $\theta$   | Curvature (radians per meter) |
| $D_{tube}$ | Body Diameter (meters)        |
| $\psi_i$   | Difference between steering direction and tendon position |

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

LENGTH = 5
CURVATURE = 0.5
BODY_DIAMETER = 0.0762 # 3 inches
TENDON_POSITION = 0

def basic_tendon_control(
    steering_direction: float,
    length: float = LENGTH,
    curvature: float = CURVATURE,
    body_diameter: float = BODY_DIAMETER,
    tendon_position: float = TENDON_POSITION
) -> float:
    return length - (curvature * (body_diameter / 2) * np.cos(tendon_position - steering_direction))

x = np.linspace(0, 2 * np.pi, 100)
plt.plot(x, basic_tendon_control(x))
plt.title("Basic Tendon Control")
plt.xlabel('Direction (radians)')
plt.ylabel('Length (meters)')
plt.grid()
plt.show()

This works great (kinda), but could be better for my application. So, by doing some math we can get this:
$$
\Delta l_i=\frac{\theta(D_{tube}/2)cos(\psi_i-\phi)}{R_{spool}}
$$
This equation gets the *difference* in tendon position, and removes the need to know the length, and includes the tendon spool diameter to get the result in radians. Its graph looks more like this:

In [ ]:
# CURVATURE = 0.05
# BODY_DIAMETER = 0.0762 # 3 inches
# TENDON_POSITION = 0
TENDON_SPOOL_RADIUS = 0.0254

def better_tendon_control(
    steering_direction: float,
    curvature: float = CURVATURE,
    body_diameter: float = BODY_DIAMETER,
    tendon_position: float = TENDON_POSITION,
    tendon_spool_radius: float = TENDON_SPOOL_RADIUS,
) -> float:
    return (curvature * body_diameter / 2 * np.cos(tendon_position - steering_direction)) / tendon_spool_radius

x = np.linspace(0, 2 * np.pi, 100)
plt.plot(x, better_tendon_control(x))
plt.title("Better Tendon Control")
plt.xlabel('Direction (radians)')
plt.ylabel('Length (meters)')
plt.grid()
plt.show()

We can see that the needed position is higher when the direction is facing towards the tendon, and lower when further away. This is close to what we need, but backwards. We can fix this by just multiplying by -1.

## Interface

Before we can use this, we have to convert the controller inputs to curvatures. We can do this using polar coordinates, where the radius is the curvature and $\theta$ is the direction. The controller outputs and x and y value from -1 to 1, so the conversion looks like this:

In [ ]:
def cartesian_to_polar(x, y) -> tuple[float, float]:
    rho = np.sqrt(x**2 + y**2)  # R
    phi = np.arctan2(y, x)  # Theta
    return (phi, rho)

x = np.linspace(-1, 1, 25)
y = np.linspace(-1, 1, 25)
x,y = np.meshgrid(x,y)

theta, r = cartesian_to_polar(x,y)

fig = plt.figure()

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(x, y, r, cmap='viridis')
ax1.set_title('Radius')

ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Radius")

ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(x, y, theta, cmap='plasma')
ax2.set_title('Theta')

ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Theta (Radians)")

plt.show()

This approach has issues where theta can jump. We can look more into this by viewing from the top.

In [ ]:
x = np.linspace(-1, 1, 25)
y = np.linspace(-1, 1, 25)
x,y = np.meshgrid(x,y)

theta, r = cartesian_to_polar(x,y)

fig = plt.figure()

ax = fig.add_subplot(121, projection='3d')
ax.view_init(elev=90, azim=-90, roll=0)
ax.plot_surface(x, y, theta, cmap='plasma')
ax.set_title('Theta')

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Theta")

plt.show()

At $(-1, 0)$, theta is $\pi$, and just after it drops to $-\pi$. This is equivalent to how $180^\circ$ is the same as $-180^\circ$

The final thing we can do is apply these values to the steering model for all 3 tendons.

In [ ]:
def pick_highest(v1, v2, v3):
    return np.maximum.reduce([v1, v2, v3])
    
x = np.linspace(-1, 1, 25)
y = np.linspace(-1, 1, 25)
x,y = np.meshgrid(x,y)

MAX_CURVE = 1.5

TENDON_1_ANGLE = 0.5 * pi
TENDON_2_ANGLE = -5 * pi / 6
TENDON_3_ANGLE = -1 * pi / 6

theta, r = cartesian_to_polar(x * MAX_CURVE, y * MAX_CURVE)

l1 = better_tendon_control(theta, r, tendon_position = TENDON_1_ANGLE)
l2 = better_tendon_control(theta, r, tendon_position = TENDON_2_ANGLE)
l3 = better_tendon_control(theta, r, tendon_position = TENDON_3_ANGLE)

fig = plt.figure()

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(x, y, l1, cmap='RdBu', alpha=0.6)
ax1.plot_surface(x, y, l2, cmap='PiYG', alpha=0.6)
ax1.plot_surface(x, y, l3, cmap='PuOr', alpha=0.6)
ax1.set_title('Tendons')

ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Length")

ax2 = fig.add_subplot(122, projection='3d')
ax2.view_init(elev=90, azim=-90, roll=0)
ax2.plot_surface(x, y, pick_highest(l1,l2,l3), cmap='managua')
ax2.set_title('Tendons (Top View)')

ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Length")

plt.show()